In [1]:
import psycopg2
import pandas as pd 
import numpy as np

In [2]:
# '127.0.0.1'
def create_connection(database, password, host="localhost",port="5432", user="postgres"):
    """"
    Description: This function create a postgresql connection object
    @ input params: database name,  password, and default values
    @ output params: create a connection
    """

    connection = None
    try:
        print('Connecting to the postgreSQL database ...')
        connection = psycopg2.connect(
            host=host, 
            port=port,
            database=database,
            user=user,
            password=password
        )
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            # connection.close()
            print('Database connection created.')
    
    return connection

In [3]:
def load_data(connection, sql_query, values=None):
    """"
    Description: This function runs the data pull from postgresql and save it as a dataframe 
    @ input params: postgresql connection object, sql query to run 
    @ output params: return a dataframe
    """
    try:
        # create a cursor 
        cursor = connection.cursor()
        cursor.execute(sql_query, values) # None
        df = pd.DataFrame(cursor.fetchall())
        df.columns = [x[0] for x in cursor.description]
        cursor.close()
        return df
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if connection is not None:
            connection.commit()
            connection.close()
            print('Database connection terminated.')

    # return df 

```sql  
--## DISTINCT vs GROUP BY
SELECT first_name, COUNT(*)
FROM actor
GROUP BY first_name;
```
<!-- Two methods: 
using `groupby` and   
using `value_count` -->

In [ ]:
schema = "public"
table_name = None
sql_select = "SELECT * FROM {}.{}".format(schema, table_name) # public.aws
# "SELECT * FROM public.actor"
conn = create_connection(database= None, password=None) ## just create the connection to postgresql
 
psg_df = load_data(connection=conn, sql_query=sql_select, values=None)

#### 01. using `.groupby()` 

In [ ]:
psg_df.head()

In [ ]:
temp = pd.DataFrame(psg_df.groupby('first_name')['first_name'].count().sort_values(ascending=False))
temp.head()
# reset_index()#sort_values(by=0, ascending=False).head(10)

#### 02. using `.value_count()`

```sql 
--##
SELECT
	customer_id,
	Total_spend,
	CASE WHEN Total_spend <= 30 THEN 'Bronze'
		 WHEN Total_spend <= 60 THEN 'Silver'
		 WHEN Total_spend <= 90 THEN 'Gold'
		 ELSE  'Diamon'
	END AS Customer_grading
FROM (
	SELECT customer_id, SUM(amount) AS Total_spend
	FROM payment
	GROUP BY customer_id
	ORDER BY SUM(amount) | 2 ) sub ;
```

In [4]:
schema = "public"
table_name = "payment"
sql_select = "SELECT * FROM {}.{}".format(schema, table_name) # public.aws
conn = create_connection("users", "postgres") ## just create the connection to postgresql 
psg_df = load_data(connection=conn, sql_query=sql_select)

Connecting to the postgreSQL database ...
Database connection created.
Database connection terminated.


In [5]:
psg_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577


In [ ]:
# float(psg_df['amount'][99])
psg_df.dtypes

In [ ]:
psg_df['amount'] = psg_df['amount'].astype(float)

In [ ]:
psg_df.dtypes

In [ ]:
psg_df['amount'][99]

In [ ]:
psg_df1 = psg_df.groupby('customer_id').sum()['amount'].reset_index().rename(columns={'amount':'Total_spend'})
psg_df1.sort_values(by=['Total_spend'], ascending=False).head()

In [ ]:
psg_df1.sort_values(by=['Total_spend'], ascending=False).tail()

In [ ]:
temp_df = psg_df1.copy()
temp_df['Customer_grading'] = temp_df['Total_spend'].map(lambda x : 'Dimond' if x >= 90 else ('Gold' if x >= 60 else ('Silver' if x >= 30 else 'Bronze') ) )
temp_df.head()

In [ ]:
temp_df[temp_df['Total_spend'] <= 45].head()

In [ ]:
def customer_grading(val):
    result = None
    if val >= 90: 
        result = "Dimond"
    elif val >= 60:
        result = "Gold"
    elif val >= 30:
        result = "Silver"
    else:
        result = "Bronze"
        
    return result

In [ ]:
customer_grading(80)

In [ ]:
temp_df['Customer_grading_2'] = temp_df['Total_spend'].map(lambda x : customer_grading(x))
temp_df.head()

#### WINDOW FUNCTIONS in Pandas

```sql
--## WINDOW FUNCTIONS
SELECT *
	--customer_id, DATE(payment_date)
	, SUM(amount) OVER (PARTITION BY customer_id ) AS Total_spend
FROM payment
ORDER BY customer_id DESC;
SELECT customer_id
	 , amount
-- 	--, DATE(payment_date)
	, SUM(amount) OVER ( ORDER BY customer_id ) AS Total_spend
	
	, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY amount) AS Row_no
	, RANK() OVER (PARTITION BY customer_id ORDER BY amount) AS Rk
	, DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY amount) AS Dr
FROM payment
--ORDER BY DATE(payment_date) DESC;
SELECT *
FROM payment;
```

In [6]:
payment_df = psg_df[['customer_id','amount']]
print(payment_df.shape)
payment_df.head()

(14596, 2)


,customer_id,amount
0,341,7.99
1,341,1.99
2,341,7.99
3,341,2.99
4,341,7.99


In [46]:
# let's get sample 10 records
temp_df = payment_df.iloc[np.r_[0:10, -10:0]]
temp_df

,customer_id,amount
0,341,7.99
1,341,1.99
2,341,7.99
3,341,2.99
4,341,7.99
5,341,5.99
6,342,5.99
7,342,5.99
8,342,2.99
9,343,4.99


#### SUM()

In [14]:
## SUM()
sum_df= temp_df.groupby(['customer_id'])['amount'].sum().reset_index().sort_values(by=['amount'], ascending=False)
sum_df

,customer_id,amount
5,341,28.95
2,252,4.99
0,245,2.99
4,264,2.99
1,251,0.99
3,263,0.99


#### Row_Number()

In [47]:
# let's first order the data 
temp_df1 = temp_df.sort_values(by=['customer_id', 'amount'], ascending=[False, False])
temp_df1

,customer_id,amount
9,343,4.99
6,342,5.99
7,342,5.99
8,342,2.99
0,341,7.99
2,341,7.99
4,341,7.99
5,341,5.99
3,341,2.99
1,341,1.99


In [33]:
temp_df1['row_number'] = temp_df1.groupby(['customer_id']).cumcount() + 1
temp_df1
#  ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY amount) AS Row_no

,customer_id,amount,row_number
9,343,4.99,1
6,342,5.99,1
7,342,5.99,2
8,342,2.99,3
0,341,7.99,1
2,341,7.99,2
4,341,7.99,3
5,341,5.99,4
3,341,2.99,5
1,341,1.99,6


#### Rank()
rank will keep the ranking, so the numbering may go 1, 2, 2, 4 etc

In [43]:
# , RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) AS Rk
temp_df1['rank']= temp_df1.groupby('customer_id')['amount'].rank(method='min', ascending=False).astype(int)
temp_df1

,customer_id,amount,rank
9,343,4.99,1
6,342,5.99,1
7,342,5.99,1
8,342,2.99,3
0,341,7.99,1
2,341,7.99,1
4,341,7.99,1
5,341,5.99,4
3,341,2.99,5
1,341,1.99,6


#### Dense_Rank()
Dense Rank will never give any gaps

In [48]:
# , DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY amount) AS Dr
temp_df1['row_number']= temp_df1.groupby('customer_id')['amount'].rank(method='first', ascending=False).astype(int)
temp_df1['rank']= temp_df1.groupby('customer_id')['amount'].rank(method='min', ascending=False).astype(int)
temp_df1['dense_rank']= temp_df1.groupby('customer_id')['amount'].rank(method='dense', ascending=False).astype(int)
temp_df1
# 341 5.99 4
# 99  1   1
# 99  1   1 
# 99  1   1  
# 98  4   2
# 98  4   2   
# 97  6   3
# 96  7   4


,customer_id,amount,row_number,rank,dense_rank
9,343,4.99,1,1,1
6,342,5.99,1,1,1
7,342,5.99,2,1,1
8,342,2.99,3,3,2
0,341,7.99,1,1,1
2,341,7.99,2,1,1
4,341,7.99,3,1,1
5,341,5.99,4,4,2
3,341,2.99,5,5,3
1,341,1.99,6,6,4


### Pandas - Assesement

In [85]:
weather_df = pd.read_csv('http://rattle.togaware.com/weatherAUS.csv')

1. Add a `day`, `month`, and `year` column (three new columns)
1. Print out the name of the columns where there are missing values
1. Drop all columns except `day`, `month`, `year`, `Rainfall`, and `Sunshine`  
1. calculate mean_rainfall 
1. Fill all missing `Rainfall` values as follows:
  1. If `Sunshine` is over 7.0, set `Rainfall` = 0
  2. Else (including if `Sunshine is NaN`) set `Rainfall` to the mean `Rainfall` value.

In [86]:
weather_df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [91]:
weather_df.MinTemp.value_counts()

 10.2    1331
 11.0    1302
 10.8    1297
 9.6     1274
 10.5    1274
         ... 
 31.9       1
-8.5        1
-7.9        1
-8.7        1
 32.5       1
Name: MinTemp, Length: 401, dtype: int64

1. First, make your initial data exploration before starting the following task.
1. Select only `location`, `MinTemp`, `MaxTemp`, `Rainfall`, `Sunshine`, `RainToday` fields that are after `2010`.
1. Rename all the columns to something easier to read and understand.
1. Then change the `RainToday` values (of cases after you renamed the column) to `0` when is `No` and `1` when it is `Yes`.
1. Add a new column `day`, `month`, and `year` column (three new columns) based on `Date` column.
1. Fill all missing `MinTemp` and `MaxTemp` values as follows:
  1. calculate the `yearlly_average` minimum and maximum temperature
  2. then fill the missing values by what you calculate as the yearly average minimum and maximum temperature.

  example: 
    - say the average `2020` temperature was `16^c`. so this means any 2020 record that missed a value would have been replaced by the yearly average minimum temperature value of `16^c`.
    - the same goes for the maximum temperature column. 
1. Finally, add the degree centigrade symbol for both the minimum and the maximum temperates columns.
  1. i.e 22.9 will be `22.9^c`
1. Lastly, create a table in your Postgresql and write this data to your database table. 

In [87]:
weather_df.head()
# first make your own data exploration before starting this task 
# I need all records after 2010 including location, MinTemp, MaxTemp, Rainfall, Sunshine, RainToday
# change RainToday values to 0 and one 
# make the min and max temps in ^c i.e 22.9 ^c
# repalce the the nan temp values to the mean value for that year


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [89]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215686 entries, 0 to 215685
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           215686 non-null  object 
 1   Location       215686 non-null  object 
 2   MinTemp        212181 non-null  float64
 3   MaxTemp        212370 non-null  float64
 4   Rainfall       209364 non-null  float64
 5   Evaporation    100873 non-null  float64
 6   Sunshine       91855 non-null   float64
 7   WindGustDir    199426 non-null  object 
 8   WindGustSpeed  199533 non-null  float64
 9   WindDir9am     198757 non-null  object 
 10  WindDir3pm     206517 non-null  object 
 11  WindSpeed9am   211564 non-null  float64
 12  WindSpeed3pm   208018 non-null  float64
 13  Humidity9am    210909 non-null  float64
 14  Humidity3pm    206849 non-null  float64
 15  Pressure9am    191819 non-null  float64
 16  Pressure3pm    191837 non-null  float64
 17  Cloud9am       122898 non-nul

In [71]:
# weather_df[pd.to_datetime(weather_df.Date).dt.year >= 2015].shape
weather_df[weather_df.Date >= '2015'].shape
# (114668, 27)

(114668, 27)

In [58]:
weather_df['year'] = pd.to_datetime(weather_df['Date']).dt.year
# weather_df[['Date','year']].head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,year
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No,2008
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No,2008
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No,2008
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No,2008
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No,2008


In [59]:
weather_df['month'] = pd.to_datetime(weather_df['Date']).dt.month
weather_df['day'] = pd.to_datetime(weather_df['Date']).dt.day
weather_df.head()
# min_temp


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,year,month,day
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,8.0,NaN,16.9,21.8,No,0.0,No,2008,12,1
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,NaN,NaN,17.2,24.3,No,0.0,No,2008,12,2
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,NaN,2.0,21.0,23.2,No,0.0,No,2008,12,3
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,NaN,NaN,18.1,26.5,No,1.0,No,2008,12,4
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,7.0,8.0,17.8,29.7,No,0.2,No,2008,12,5


In [65]:
weather_df.columns[weather_df.isnull().any()]

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [72]:
weather_df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow', 'year', 'month',
       'day'],
      dtype='object')

In [73]:
imp_cols = ['Date','Sunshine','Rainfall','day','month','year'] # Humidity9am
imp_df = weather_df[imp_cols]
imp_df.head()

,Date,Sunshine,Rainfall,day,month,year
0,2008-12-01,NaN,0.6,1,12,2008
1,2008-12-02,NaN,0.0,2,12,2008
2,2008-12-03,NaN,0.0,3,12,2008
3,2008-12-04,NaN,0.0,4,12,2008
4,2008-12-05,NaN,1.0,5,12,2008


In [75]:
meanrainfall = np.round(imp_df.Rainfall.mean(),3)
meanrainfall

2.331

In [ ]:
# 1. Fill all missing `Rainfall` values as follows:
#   1. If `Sunshine` is over 7.0, set `Rainfall` = 0
#   2. Else (including if `Sunshine is NaN`) set `Rainfall` to the mean `Rainfall` value.

In [84]:
imp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215686 entries, 0 to 215685
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Date      215686 non-null  object 
 1   Sunshine  91855 non-null   float64
 2   Rainfall  209364 non-null  float64
 3   day       215686 non-null  int64  
 4   month     215686 non-null  int64  
 5   year      215686 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 9.9+ MB


In [79]:
imp_df.shape

(215686, 6)